In [97]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from textblob import TextBlob
from scipy.sparse import csr_matrix
from nltk.corpus import words

english_corpus = set(words.words())

matrix = pd.read_csv('matrix.csv')
#matrix = matrix.iloc[:, :100]
#pd.DataFrame(list(matrix.columns), columns = ['feature']).to_csv('matrix_columns.csv', index = False)
matrix

,aa,aaa,aaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaand,aaas,aab,aabad,aabout,aaca,...,zurich,zvun,zwaar,zwkmjkkz,zxcap,zxje,zxlo,zy,zz,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from tqdm import tqdm
import numpy as np

from textblob import Word

def spellcheck(word, threshold):
    value = Word(word).spellcheck()[0]
    if (value[1] < threshold):
        return f'-1'
    return value[0]


def correct(text):
    return TextBlob(text).correct().string

np_spellcheck = np.vectorize(spellcheck)

def process_words(columns):
    word_list = [(word, spellcheck(word, 0.7)) for word in columns]
    word_list = dict(word_list)
    return word_list
    


In [7]:
columns_df = pd.DataFrame(list(matrix.columns), columns = ['feature'])
columns_df

,feature
0,aa
1,aaa
2,aaaaaaaa
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
4,aaaaand
...,...
29223,zxje
29224,zxlo
29225,zy
29226,zz


In [8]:
tqdm.pandas(desc="feature_spellcheck")
%time columns_df['feature_spellcheck'] = columns_df['feature'].progress_apply(lambda word: spellcheck(word, 0.7))
columns_df

feature_spellcheck: 100%|██████████| 29228/29228 [1:04:35<00:00,  7.54it/s]

CPU times: user 1h 2min 59s, sys: 21.5 s, total: 1h 3min 20s
Wall time: 1h 4min 35s


,feature,feature_spellcheck
0,aa,-1
1,aaa,-1
2,aaaaaaaa,-1
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,-1
4,aaaaand,-1
...,...,...
29223,zxje,-1
29224,zxlo,-1
29225,zy,by
29226,zz,-1


In [98]:
print(matrix.shape)
matrix.head()

(9, 29228)


,aa,aaa,aaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaand,aaas,aab,aabad,aabout,aaca,...,zurich,zvun,zwaar,zwkmjkkz,zxcap,zxje,zxlo,zy,zz,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
matrix = matrix.drop(columns_df.loc[columns_df['feature_spellcheck'] == "-1"]['feature'].values, axis = 1)

print(matrix.shape)
matrix.head()

(9, 17658)


,aaas,aab,aabad,aabout,aah,aaron,ab,abad,abadan,abandomns,...,zpst,zqf,zsfuck,zucula,zud,zurich,zvun,zwaar,zxcap,zy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
renamed_columns = dict(columns_df.loc[columns_df['feature_spellcheck'] != "-1"].to_dict('split')['data'])


In [101]:
matrix = matrix.rename(columns = renamed_columns)
matrix.head()

,alas,cab,bad,about,aah,aaron,ab,bad,abandon,abandons,...,zest,of,stuck,uvula,mud,rich,even,war,cap,by
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
for column in tqdm(matrix.columns[matrix.columns.duplicated()]):
    matrix[f'{column}_duplicated'] = matrix[column].groupby(level = 0, axis = 1).sum()

matrix = matrix.drop(matrix.columns[matrix.columns.duplicated()], axis = 1)
print(matrix.shape)
matrix.head()

100%|██████████| 6219/6219 [00:23<00:00, 268.24it/s]

(9, 11439)


,aah,aaron,ab,abandons,abate,abatement,battre,abdomen,abdominal,abduct,...,polite_duplicated,villa_duplicated,million_duplicated,piti_duplicated,zu_duplicated,milk_duplicated,zone_duplicated,room_duplicated,pope_duplicated,uvula_duplicated
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
matrix.rename(columns=lambda x: x.replace('_duplicated', ''), inplace=True)
print(matrix.shape)
matrix.head()

(9, 11439)


,aah,aaron,ab,abandons,abate,abatement,battre,abdomen,abdominal,abduct,...,polite,villa,million,piti,zu,milk,zone,room,pope,uvula
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
matrix = matrix.applymap(lambda n: 1 if n > 1 else n)